In [0]:
# Databricks notebook source
import time
from pyspark.sql.functions import col
start = time.time()
import datetime
import os
import re
import shutil

In [0]:
dbutils.widgets.text("from_date", "")
from_date = dbutils.widgets.get("from_date")

dbutils.widgets.text("to_date", "")
to_date = dbutils.widgets.get("to_date")

dbutils.widgets.text("country_code", "")
country_code = dbutils.widgets.get("country_code") 

dbutils.widgets.text("region_name", "")
region_name = dbutils.widgets.get("region_name")

dbutils.widgets.text("product_sku", "")
product_sku = dbutils.widgets.get("product_sku")

dbutils.widgets.text("email_from", "")
email_from = dbutils.widgets.get("email_from")

dbutils.widgets.text("email_to", "")
email_to = dbutils.widgets.get("email_to")

dbutils.widgets.text("email_cc", "")
email_cc = dbutils.widgets.get("email_cc")

dbutils.widgets.text("dir_tmp", "")
dir_tmp = dbutils.widgets.get("dir_tmp")

dbutils.widgets.text("env", "")
env = dbutils.widgets.get("env")

# NEW WIDGET: To pass the SendGrid API Key securely without using Secret Manager
dbutils.widgets.text("sendgrid_api_key", "", "SendGrid API Key")
sendgrid_api_key = dbutils.widgets.get("sendgrid_api_key")

In [0]:
# Define parameters
current_date = datetime.datetime.now()
execution_id = current_date.strftime('%Y-%m-%d')

# Date range logic
if from_date and to_date:
    # Use provided dates
    final_from_date = from_date
    final_to_date = to_date
    print(f"Using provided date range: {final_from_date} to {final_to_date}")
else:
    # Default to previous calendar month
    # Get first day of current month
    first_day_current_month = current_date.replace(day=1) # 16 Oct --> 01 Oct
    
    # Get last day of previous month
    last_day_prev_month = first_day_current_month - datetime.timedelta(days=1) # 30 Sept
    
    # Get first day of previous month
    first_day_prev_month = last_day_prev_month.replace(day=1) # 01 Sept
    
    final_from_date = first_day_prev_month.strftime('%Y-%m-%d')
    final_to_date = last_day_prev_month.strftime('%Y-%m-%d')
    print(f"No dates provided. Defaulting to previous month: {final_from_date} to {final_to_date}")

# Use the actual date range for reporting labels
report_start = datetime.datetime.strptime(final_from_date, '%Y-%m-%d')
report_end = datetime.datetime.strptime(final_to_date, '%Y-%m-%d')

# Update report labels to reflect the actual date range
if report_start.month == report_end.month and report_start.year == report_end.year:
    # Same month
    report_month = report_start.strftime('%B %Y')
    report_year_month = report_start.strftime('%Y-%m')
else:
    # Date range spans multiple months
    report_month = f"{report_start.strftime('%B %Y')} to {report_end.strftime('%B %Y')}"
    report_year_month = f"{report_start.strftime('%Y-%m')}_to_{report_end.strftime('%Y-%m')}"

csv_filename = f"Regional_sales_Report_{report_year_month}.csv"

# Define DBFS base path
base_dir = dir_tmp
temp_dir = f"{base_dir}/{execution_id}"
archive_dir = f"{base_dir}/archive"
temp_output = f"{temp_dir}/output"
temp_csv = f"{temp_dir}/{csv_filename}"

# Delete old temporary files
print(f"Cleaning up old files in temp directory: {temp_dir}")
dbutils.fs.rm(temp_dir, True)

# Create directory in DBFS
dbutils.fs.mkdirs(temp_dir)
dbutils.fs.mkdirs(archive_dir)

In [0]:
%python
print("Extracting dealer enrollment data...")

query = f"""
SELECT
    country_code,
    region_name,
    product_name,
    customer_segment,
    sales_channel,
    DATE_FORMAT(sale_date, 'yyyy-MM') AS year_month,
    'Sale' AS transaction_category,
    SUM(quantity) AS total_quantity,
    SUM(total_revenue) AS total_revenue
FROM
    alldatatech_project.sales_reporting.regional_sales_data
WHERE
    country_code = '{country_code}'
    AND region_name = '{region_name}'
    AND product_sku = '{product_sku}'
    AND sale_date BETWEEN '{final_from_date}' AND '{final_to_date}'
    AND quantity > 0
    AND shipment_status IN ('Delivered', 'Shipped')
GROUP BY
    country_code, region_name, product_name, customer_segment, sales_channel, DATE_FORMAT(sale_date, 'yyyy-MM')

UNION ALL

SELECT
    country_code,
    region_name,
    product_name,
    customer_segment,
    sales_channel,
    DATE_FORMAT(sale_date, 'yyyy-MM') AS year_month,
    'Return' AS transaction_category,
    SUM(quantity) AS total_quantity,
    SUM(total_revenue) AS total_revenue
FROM
    alldatatech_project.sales_reporting.regional_sales_data
WHERE
    country_code = '{country_code}'
    AND region_name = '{region_name}'
    AND product_sku = '{product_sku}'
    AND sale_date BETWEEN '{final_from_date}' AND '{final_to_date}'
    AND (quantity < 0 OR shipment_status IN ('Cancelled', 'Returned'))
GROUP BY
    country_code, region_name, product_name, customer_segment, sales_channel, DATE_FORMAT(sale_date, 'yyyy-MM')

ORDER BY
    year_month, transaction_category, total_revenue DESC;
"""

result_df = spark.sql(query)
display(result_df)

total_records = result_df.count()
Sale_count = result_df.filter(col("transaction_category") == "Sale").count()
Return_count = result_df.filter(col("transaction_category") == "Return").count()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2024913250977132>, line 17
      1 print("Extracting dealer enrollment data...")
      3 query = f"""
      4 SELECT
      5     country_code,
      6     region_name,
      7     product_name,
      8     customer_segment,
      9     sales_channel,
     10     DATE_FORMAT(sale_date, 'yyyy-MM') AS year_month,
     11     'Sale' AS transaction_category,
     12     SUM(quantity) AS total_quantity,
     13     SUM(total_revenue) AS total_revenue
     14 FROM
     15     uc_sp.sales_reporting.regional_sales_data
     16 WHERE
---> 17     country_code = '{country_code}'
     18     AND region_name = '{region_name}'
     19     AND product_sku = '{product_sku}'
     20     AND sale_date BETWEEN '2025-11-01' AND '2025-11-30'
     21     AND quantity > 0
     22     AND shipment_status IN ('Delivered', 'Shipped')
     23 GROUP B

In [0]:
print("Writing result to CSV...")

# Write to output directory
result_df.coalesce(1).write.mode("overwrite").option("header", "true").csv(temp_output)

# Remove non-part files - Replaced missing shared_functions.remove_non_part_files
print(f" Removing non-essential files from {temp_output}")
output_files = dbutils.fs.ls(temp_output)
for f in output_files:
    # Retain the single 'part-' file and the '_SUCCESS' file
    if not (f.name.startswith("part-") or f.name == "_SUCCESS"):
        # Delete other metadata/temp files
        print(f"  Removing non-essential file: {f.path}")
        dbutils.fs.rm(f.path)

# List files and locate the part file
files = dbutils.fs.ls(temp_output)
part_file = next((f.path for f in files if f.name.startswith("part-") and f.name.endswith(".csv")), None)

if not part_file:
    raise Exception("No part CSV file found after writing.")

# Move part file to final filename
dbutils.fs.mv(part_file, temp_csv)
print(f"CSV saved as: {temp_csv}")

In [0]:
# NOTE: The function now accepts 'api_key' as an argument, replacing the secret manager call.
def send_email(from_email, to_email, subject, message, env, cc_email='', body_type='', attachment_path=None, attachment_name=None, api_key=None):
    import ssl
    import smtplib
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email.mime.multipart import MIMEMultipart
    from email import encoders

    if not api_key:
        raise ValueError("API Key is missing. Please provide the SendGrid API Key.")
        
    html = True if body_type.lower() == 'html' else False
    sendgrid_api_key = api_key # CHANGED: Uses the API key passed as an argument

    # Use ssl.create_default_context() for a client connection.
    context = ssl.create_default_context()
    context.minimum_version = ssl.TLSVersion.TLSv1_2

    # Create a multipart message and set headers
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = ",".join([email.strip() for email in re.split(pattern='[,;]', string=to_email)])
    msg['Cc'] = ",".join([email.strip() for email in re.split(pattern='[,;]', string=cc_email)]) if cc_email else ''
    msg['Subject'] = subject

    # Prepare the list of all recipients for smtplib.sendmail()
    to_email_list = re.split(pattern='[,;]', string=to_email) + (re.split(pattern='[,;]', string=cc_email) if cc_email else [])

    # Attach body
    msg.attach(MIMEText(message, 'html')) if html else msg.attach(MIMEText(message, 'plain'))

    # Attach file if provided
    if attachment_path and attachment_name:
        # Read the file from DBFS
        with open(attachment_path.replace("dbfs:/", "/dbfs/"), "rb") as attachment:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
        
        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part)
        
        # Add header with the file name
        part.add_header(
            'Content-Disposition',
            f'attachment; filename= {attachment_name}',
        )
        
        msg.attach(part)
        print(f"  Attached file: {attachment_name}")

    # Create SMTP session
    with smtplib.SMTP_SSL("smtp.sendgrid.net", 465, context=context) as server:
        server.login(user='apikey', password=sendgrid_api_key)
        server.sendmail(from_email, to_email_list, msg.as_string())

In [0]:
print("Preparing to send email...")
try:
    file_info = dbutils.fs.ls(temp_csv)[0]
    file_size_mb = file_info.size / (1024 * 1024)
except Exception as e:
    print(f"Error reading file size: {e}")
    file_size_mb = 0.01 # Default small size if unable to read

# Check file size against SendGrid limits
MAX_ATTACHMENT_SIZE_MB = 25  # Stay under 30 MB limit with buffer

if file_size_mb > MAX_ATTACHMENT_SIZE_MB:
    print(f"WARNING: File size ({file_size_mb:.2f} MB) exceeds recommended limit!")
    print("Consider uploading to a shared location and sending a link instead.")

email_subject = f"Regional_sales Extract Report - {report_month}"

# -----------------------------------------------------
# --- MODERNIZED EMAIL BODY SECTIONS (Environment Warnings Removed) ---
# -----------------------------------------------------

# Common Styles for modern, centered look (Applied to the outer wrapper)
email_style = """
style="
    font-family: 'Ford Antenna', Arial, sans-serif; 
    -webkit-font-smoothing: antialiased;
    color: #333;
    line-height: 1.6;
    background-color: #f7f9fc; 
    padding: 20px;
"
"""
env_banner = ""
    
# 2. Header and Introduction (Modified to remove the environment tag)
header_intro_section = f"""
<div style="padding: 25px 30px; background-color: #00095B; color: white; border-radius: 8px 8px 0 0;">
    <h1 style="font-size: 28px; font-weight: 700; margin: 0;">Regional_sales Extract Report</h1>
    <p style="font-size: 15px; margin: 5px 0 0 0; opacity: 0.8;">
        Regional Sales Report for **{final_from_date} to {final_to_date}**
    </p>
</div>
<div style="padding: 30px; background-color: #ffffff; border-bottom: 1px solid #eee;">
    <p style="margin-bottom: 15px; font-size: 15px;">Hello,</p>
    <p style="margin-bottom: 0; font-size: 15px;">
        The scheduled **Regional_sales Extract Report** has been successfully generated and is ready for your review.
    </p>
</div>
"""

# 3. HTML Table (Unchanged)
html_table = f"""
<div style="padding: 30px 30px 20px 30px; background-color: #ffffff;">
    <h2 style="color: #00095B; margin-bottom: 20px; font-size: 22px; font-weight: 700;">📊 Report Summary</h2>
    <table role="presentation" width='100%' border='0' cellpadding='0' cellspacing='0' style='border-collapse: collapse; border-radius: 8px; overflow: hidden; border: 1px solid #ddd;'>
        <thead>
            <tr style='background-color: #f0f4f7;'>
                <th style='color: #00095B; text-align: left; font-size: 14px; font-weight: 700; padding: 12px 15px; width: 50%; border-bottom: 2px solid #ddd;'>Metric</th>
                <th style='color: #00095B; text-align: right; font-size: 14px; font-weight: 700; padding: 12px 15px; width: 50%; border-bottom: 2px solid #ddd;'>Value</th>
            </tr>
        </thead>
        <tbody>
            <tr style='background-color: #ffffff;'>
                <td style='padding: 12px 15px; border-bottom: 1px solid #eee; color: #555; font-weight: 500; font-size: 14px;'>📅 Report Period</td>
                <td style='padding: 12px 15px; border-bottom: 1px solid #eee; color: #00095B; font-weight: bold; font-size: 15px; text-align: right;'>{report_month}</td>
            </tr>
            <tr style='background-color: #f9f9f9;'>
                <td style='padding: 12px 15px; border-bottom: 1px solid #eee; color: #555; font-weight: 500; font-size: 14px;'>📆 Date Range</td>
                <td style='padding: 12px 15px; border-bottom: 1px solid #eee; color: #00095B; font-weight: bold; font-size: 15px; text-align: right;'>{final_from_date} to {final_to_date}</td>
            </tr>
            
            <tr style='background-color: #e3f2fd; border-left: 4px solid #2196f3;'>
                <td style='padding: 15px; border-bottom: 1px solid #bbdefb; color: #333; font-weight: 600; font-size: 16px;'>📋 Total Records</td>
                <td style='padding: 15px; border-bottom: 1px solid #bbdefb; color: #1565c0; font-weight: bold; font-size: 20px; text-align: right;'>{total_records:,}</td>
            </tr>
            <tr style='background-color: #e8f5e9; border-left: 4px solid #4caf50;'>
                <td style='padding: 12px 15px; border-bottom: 1px solid #c8e6c9; color: #333; font-weight: 500; font-size: 14px;'>✅ Return Count</td>
                <td style='padding: 12px 15px; border-bottom: 1px solid #c8e6c9; color: #2e7d32; font-weight: bold; font-size: 16px; text-align: right;'>{Return_count:,}</td>
            </tr>
            <tr style='background-color: #ffffff; border-left: 4px solid #ffc107;'>
                <td style='padding: 12px 15px; border-bottom: 1px solid #eee; color: #333; font-weight: 500; font-size: 14px;'>🚀 Sale Count</td>
                <td style='padding: 12px 15px; border-bottom: 1px solid #eee; color: #e65100; font-weight: bold; font-size: 16px; text-align: right;'>{Sale_count:,}</td>
            </tr>
            <tr style='background-color: #f9f9f9;'>
                <td style='padding: 12px 15px; color: #555; font-weight: 500; font-size: 14px;'>💾 File Size</td>
                <td style='padding: 12px 15px; color: #00095B; font-weight: bold; font-size: 15px; text-align: right;'>{file_size_mb:.2f} MB</td>
            </tr>
        </tbody>
    </table>
</div>
"""

# 4. Attachment/Action Section 
attachment_action = f"""
<div style="padding: 20px 30px; background-color: #f0f4f7; border-top: 1px solid #ddd; border-radius: 0 0 8px 8px;">
    <p style="font-size: 16px; color: #00095B; margin: 0 0 10px 0; font-weight: bold;">
        <strong style="color: #00095B;">📥 Download Attachment</strong>
    </p>
    <p style="font-size: 14px; color: #555; margin: 0 0 15px 0;">
        The detailed report is attached as: <code style="background-color: #e3e8ed; padding: 2px 5px; border-radius: 3px; color: #333; font-family: monospace;">{csv_filename}</code>
    </p>
    <a href="#" style="background-color: #00095B; color: white; padding: 10px 20px; text-decoration: none; border-radius: 4px; font-weight: bold; display: inline-block; font-size: 14px;">
        View Report Details
    </a>
</div>
"""

# 5. Signature 
signature = f"""
<div style="margin-top: 40px; padding: 20px; border-top: 1px solid #ddd; text-align: center; color: #777;">
    <p style="font-size: 14px; margin: 0;">Thank you,</p>
    <p style="font-size: 15px; color: #00095B; font-weight: bold; margin: 5px 0 0 0;">Sales Report Automation Team</p>
    <p style="font-size: 12px; color: #999; margin: 8px 0 0 0;">(This is an automated email. Please do not reply.)</p>
</div>
"""

# Combine all sections within a main wrapper
email_body = f"""
<div {email_style}>
    <div style="max-width: 600px; margin: 0 auto; box-shadow: 0 10px 25px rgba(0,0,0,0.1); border-radius: 8px; background-color: #ffffff;">
        {env_banner}
        {header_intro_section}
        {html_table}
        {attachment_action}
    </div>
    {signature}
</div>
"""

# Send email with attachment
send_email(
    from_email=email_from,
    to_email=email_to,
    subject=email_subject,
    message=email_body,
    env=env,
    cc_email=email_cc,
    body_type='html',
    attachment_path=temp_csv,
    attachment_name=csv_filename,
    api_key=sendgrid_api_key
)

# Copy file to archive
print("Archiving the report file...")
dbutils.fs.cp(temp_csv, f"{archive_dir}/{csv_filename}")

print(f"Report archived to: {archive_dir}/{csv_filename}")
print(f"Total execution time: {(time.time() - start):.2f} seconds")

In [0]:
print(f"Cleaning up temp_dir: {temp_dir}")
dbutils.fs.rm(temp_dir, recurse=True)
print("Cleanup complete.")